My first objective is to create a Pydantic model to validate user input data in a fictional customer support system.
I'll use venv to create a virtual environment .venv into which I'll install the IPkernel and Pydantic packages (it's a best practice and will prevent any conflicts with packages installed in the global environment). You'll also need to `pip install pydantic[email]` - the email validator.

In [2]:
# Import the pydantic and json libraries
from pydantic import BaseModel, ValidationError, EmailStr
import json

## Define a Pydantic model for user input and populate it with some sample data

In [4]:
# Create a Pydantic model to validate user input data consisting of a customer_number (int), email (str), and issue_description (str).
class UserInput(BaseModel):
    customer_number: int
    email: EmailStr
    issue_description: str
    
# Sample data to populate the model
sample_data = {
    "customer_number": 12345,
    "email": "  user@example.com  ",
    "issue_description": "I am unable to access my account."
}

Validate the sample data

In [5]:
# Validate sample_data

user_input = UserInput(**sample_data)

print(user_input)

customer_number=12345 email='user@example.com' issue_description='I am unable to access my account.'


Create an error on purpose - string instead of number and no email

In [ ]:
# Populate model with invalid data to see validation errors
invalid_data = {
    "customer_number": "not_a_number",
    "email": "invalid_email",
    "issue_description": 6
}

#Run this cell below to see validation errors returned as JSON
try:
    user_input = UserInput(**invalid_data)
except ValidationError as e:
    print(e.json()) 

[{"type":"int_parsing","loc":["customer_number"],"msg":"Input should be a valid integer, unable to parse string as an integer","input":"not_a_number","url":"https://errors.pydantic.dev/2.11/v/int_parsing"},{"type":"value_error","loc":["email"],"msg":"value is not a valid email address: An email address must have an @-sign.","input":"invalid_email","ctx":{"reason":"An email address must have an @-sign."},"url":"https://errors.pydantic.dev/2.11/v/value_error"},{"type":"string_type","loc":["issue_description"],"msg":"Input should be a valid string","input":6,"url":"https://errors.pydantic.dev/2.11/v/string_type"}]


In [7]:
# Create a function that takes a dictionary as input, validates it against the UserInput model, and returns either the validated data or the validation errors.
def validate_user_input(data: dict):
    try:
        user_input = UserInput(**data)
        print(f"✅ Valid user input created:")
        print(f"{user_input.model_dump_json(indent=2)}")
        return user_input
    except ValidationError as e:
        print(f"❌ Validation errors found:")
        print(f"{e.json(indent=2)}")
        return e.json()

In [8]:
#Try to validate invalid data
validate_user_input(invalid_data)

❌ Validation errors found:
[
  {
    "type": "int_parsing",
    "loc": [
      "customer_number"
    ],
    "msg": "Input should be a valid integer, unable to parse string as an integer",
    "input": "not_a_number",
    "url": "https://errors.pydantic.dev/2.11/v/int_parsing"
  },
  {
    "type": "value_error",
    "loc": [
      "email"
    ],
    "msg": "value is not a valid email address: An email address must have an @-sign.",
    "input": "invalid_email",
    "ctx": {
      "reason": "An email address must have an @-sign."
    },
    "url": "https://errors.pydantic.dev/2.11/v/value_error"
  },
  {
    "type": "string_type",
    "loc": [
      "issue_description"
    ],
    "msg": "Input should be a valid string",
    "input": 6,
    "url": "https://errors.pydantic.dev/2.11/v/string_type"
  }
]


'[{"type":"int_parsing","loc":["customer_number"],"msg":"Input should be a valid integer, unable to parse string as an integer","input":"not_a_number","url":"https://errors.pydantic.dev/2.11/v/int_parsing"},{"type":"value_error","loc":["email"],"msg":"value is not a valid email address: An email address must have an @-sign.","input":"invalid_email","ctx":{"reason":"An email address must have an @-sign."},"url":"https://errors.pydantic.dev/2.11/v/value_error"},{"type":"string_type","loc":["issue_description"],"msg":"Input should be a valid string","input":6,"url":"https://errors.pydantic.dev/2.11/v/string_type"}]'

In [9]:
#Create invalid data with a missing field
incomplete_data = {
    "customer_number": 67890,
    "issue_description": "My order hasn't arrived yet."
}

In [10]:
# Try validating the incomplete data
validate_user_input(incomplete_data)

❌ Validation errors found:
[
  {
    "type": "missing",
    "loc": [
      "email"
    ],
    "msg": "Field required",
    "input": {
      "customer_number": 67890,
      "issue_description": "My order hasn't arrived yet."
    },
    "url": "https://errors.pydantic.dev/2.11/v/missing"
  }
]


'[{"type":"missing","loc":["email"],"msg":"Field required","input":{"customer_number":67890,"issue_description":"My order hasn\'t arrived yet."},"url":"https://errors.pydantic.dev/2.11/v/missing"}]'